In [1]:
    #from sklearn.model_selection import train_test_split
#from sklearn.model_selection import ParameterGrid

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import IsolationForest

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree

#from sklearn_pandas import DataFrameMapper
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
#from sklearn.metrics import PredictionErrorDisplay

from sklearn.decomposition import PCA

#from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import datetime
import seaborn as sns
sns.set_palette('muted')
sns.set_color_codes('muted')
sns.set_style('white')

import warnings
warnings.filterwarnings('ignore')

from statsmodels.graphics.mosaicplot import mosaic
pd.set_option('display.max_columns', 70)

**geo data**

In [2]:
%%time
xls = pd.ExcelFile('C:/Users/elysi/Tele2_project/data/geography/yandex_objects_Sanya_version_2.xlsx')
df_yandex = pd.read_excel(xls, 'Итог')
df_sub_categories = pd.read_excel(xls, 'Выбранные категории')
df_categories = pd.read_excel(xls, 'Категории')

CPU times: total: 2min 40s
Wall time: 2min 43s


In [3]:
df_sub_categories.shape, df_categories.shape

((558, 3), (58, 1))

In [4]:
#old - (594290, 11)
df_yandex.shape

(594290, 11)

In [5]:
df_yandex.head(5)

,id,coordinate,lon,lat,title,infos,category,Подкатегория,Категория,Для дупликатов,h3_9_yandex
0,26689477,"30.16816, 59.8155",30.168160,59.815500,Емг Северо-Запад,"Рабочая ул., 7, Красносельский район, историче...",742,Шиномонтаж,Авто,"Емг Северо-ЗападРабочая ул., 7, Красносельский...",89110610b7bffff
1,23888554,"30.167744, 59.815758",30.167744,59.815758,Мосавтошина,"Рабочая ул., 7Л, Красносельский район, историч...",742,Шиномонтаж,Авто,"МосавтошинаРабочая ул., 7Л, Красносельский рай...",89110610b7bffff
2,23856942,"30.142624, 59.825364",30.142624,59.825364,Икар,"просп. Народного Ополчения, 86<br>+7 (812) 643...",742,Шиномонтаж,Авто,"Икарпросп. Народного Ополчения, 86<br>+7 (812)...",89110610b97ffff
3,23802964,"30.152274, 59.822647",30.152274,59.822647,ProAutoSPB,"12, территория Станционный Посёлок, Санкт-Пете...",742,Шиномонтаж,Авто,"ProAutoSPB12, территория Станционный Посёлок, ...",89110610b13ffff
4,25525679,"30.173387, 59.824735",30.173387,59.824735,PitStop,"ул. Партизана Германа, 47, Санкт-Петербург<br>...",742,Шиномонтаж,Авто,"PitStopул. Партизана Германа, 47, Санкт-Петерб...",89110610b2bffff


In [6]:
df_sub_categories[df_sub_categories['Отдельно'] == 1].head(5)

,Подкатегория,Отдельно,Категория
93,Кинотеатры,1.0,Кинотеатры
104,Торговые центры,1.0,Торговые центры
111,Больницы,1.0,Больницы
123,Детские сады,1.0,Детские сады
125,Вузы,1.0,Вузы


In [7]:
df_categories.head(5)

,Категория
0,Авто
1,Безопасность
2,Бизнес
3,Государство
4,Домашние животные


**select only needed categories and sub categories**

In [8]:
dict = {'Подкатегория': 'sub_category'}
 
df_yandex.rename(columns=dict,
                          inplace=True)

In [9]:
filtered_yandex_subcategories = df_yandex[
    (df_yandex['sub_category'].isin(df_sub_categories[df_sub_categories['Отдельно'] == 1]['Подкатегория']))
]

In [10]:
filtered_yandex_subcategories.shape

(33125, 11)

**basic SPB hexagons**

In [432]:
df_time = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_time_based_features_full.csv')

In [433]:
df_base_hes = df_time[['h3_9']]

In [434]:
df_base_hes.shape

(3892, 1)

In [451]:
new_h3_9_values = df_yandex_categories[~df_yandex_categories['h3_9_yandex'].isin(df_base_hes['h3_9'])][['h3_9_yandex']]
dict = {'h3_9_yandex': 'h3_9'}
 
new_h3_9_values.rename(columns=dict,
                          inplace=True)

# Append the new entries to df_metro_agr
df_base_hes_full = pd.concat([df_base_hes, new_h3_9_values], ignore_index=True)

In [452]:
df_base_hes_full.shape

(7850, 1)

In [453]:
df_base_hes_full

,h3_9
0,8911060006bffff
1,89110600077ffff
2,89110600113ffff
3,89110600127ffff
4,89110600183ffff
...,...
7845,891106b9c5bffff
7846,891106a8583ffff
7847,891106a35bbffff
7848,891106a3577ffff


## agregate hexagons: categories

In [11]:
df_yandex.shape

(594290, 11)

In [12]:
filtered_yandex_categories = df_yandex.groupby('Категория').apply(lambda x: x.drop_duplicates(subset='Для дупликатов', keep='first')).reset_index(drop=True)

In [13]:
filtered_yandex_categories['Для дупликатов'].nunique()

272680

In [14]:
filtered_yandex_categories.shape

(368924, 11)

In [15]:
filtered_yandex_categories = df_yandex[['h3_9_yandex', 'Категория']]

In [16]:
dict = {'Категория': 'category'}
 
filtered_yandex_categories.rename(columns=dict,
                          inplace=True)

In [17]:
filtered_yandex_categories.head(5)

,h3_9_yandex,category
0,89110610b7bffff,Авто
1,89110610b7bffff,Авто
2,89110610b97ffff,Авто
3,89110610b13ffff,Авто
4,89110610b2bffff,Авто


In [18]:
filtered_yandex_categories.category.nunique()

58

In [19]:
%%time
unique_objects = filtered_yandex_categories['category'].unique()
df_yandex_categories = pd.DataFrame(index=df_yandex['h3_9_yandex'].unique())

for obj in unique_objects:
    df_yandex_categories[f'{obj}_count'] = filtered_yandex_categories[filtered_yandex_categories['category'] == obj].groupby('h3_9_yandex')['category'].count()

df_yandex_categories = df_yandex_categories.fillna(0).astype(int)
df_yandex_categories.insert(0, 'h3_9_yandex', df_yandex_categories.index)
df_yandex_categories = df_yandex_categories.reset_index(drop=True)

CPU times: total: 3.62 s
Wall time: 3.69 s


In [20]:
for column in df_yandex_categories.columns:
    if column not in ['h3_9_yandex']:
        df_yandex_categories.rename(columns={column: 'category_' + column}, inplace=True)

In [21]:
df_yandex_categories.shape

(7755, 59)

In [22]:
df_yandex_categories.head(3)

,h3_9_yandex,category_Авто _count,category_Безопасность _count,category_Бизнес _count,category_Государство _count,category_Домашние животные _count,category_Дом и интерьер _count,category_Еда и напитки _count,category_Компьютеры и интернет _count,category_Красота и здоровье _count,category_Культура и искусство _count,category_Кинотеатры_count,category_Магазины _count,category_Торговые центры_count,category_Медицина и фармацевтика _count,category_Больницы_count,category_Наука и образование _count,category_Детские сады_count,category_Вузы_count,category_Недвижимость _count,category_Бизнес-центры_count,category_Общежития_count,category_Общество _count,category_Одежда и мода _count,category_Музеи_count,category_Отдых и развлечения _count,category_Зоопарки_count,category_Дельфинарии и океанариумы_count,category_Развлекательные центры_count,category_Производство и поставки _count,category_Реклама и полиграфия _count,category_Спорт и фитнес _count,category_Справочно-информационные системы _count,category_Средства массовой информации _count,category_Строительство и ремонт _count,category_Телекоммуникации и связь _count,category_Товары для детей _count,category_Транспорт и перевозки _count,category_Железнодорожные вокзалы_count,category_Автовокзалы_count,category_Аэропорты_count,category_Гостиницы_count,category_Туризм _count,category_Услуги _count,category_Финансы _count,category_Автостоянки и паркинги_count,category_Бесплатные парковки_count,category_Парковки_count,category_Платные парковки_count,category_Подземные парковки_count,category_Поликлиники_count,category_Гимназии_count,category_Колледжи_count,category_Лицеи_count,category_Общеобразовательные школы_count,category_Планетарии_count,category_Парки аттракционов_count,category_Парки культуры и отдыха_count,category_Производственные предприятия_count
0,89110610b7bffff,22,1,3,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5,0,1,0,0,17,0,0,0,0,0,0,1,0,4,0,1,1,1,0,0,0,0,0,0,0,0,0,0,2
1,89110610b97ffff,40,1,8,0,9,7,20,8,59,3,0,7,0,9,0,1,0,0,0,0,0,0,2,0,4,1,0,1,0,1,8,0,0,0,2,23,0,0,0,0,0,0,7,5,3,0,2,0,0,0,0,0,0,0,0,0,0,0
2,89110610b13ffff,41,3,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [126]:
df_yandex_categories.filter(regex = 'спорт|Спорт').columns

Index(['category_Спорт и фитнес _count', 'category_Транспорт и перевозки _count'], dtype='object')

In [127]:
df_yandex_categories = df_yandex_categories.drop(['category_Спорт и фитнес _count'], axis = 1)

In [129]:
unique_attraction_categories = ['category_Кинотеатры_count', 'category_Музеи_count',
                               'category_Дельфинарии и океанариумы_count', 'category_Парки культуры и отдыха_count',
                               'category_Торговые центры_count']

In [130]:
df_yandex_categories = df_yandex_categories.drop(unique_attraction_categories, axis = 1)

In [131]:
rename_dict = {
    'category_Авто _count': 'category_auto_count',
    'category_Безопасность _count': 'category_security_count',
    'category_Бизнес _count': 'category_business_count',
    'category_Государство _count': 'category_government_count',
    'category_Домашние животные _count': 'category_pets_count',
    'category_Дом и интерьер _count': 'category_home_and_interior_count',
    'category_Еда и напитки _count': 'category_food_and_drinks_count',
    'category_Компьютеры и интернет _count': 'category_computers_and_internet_count',
    'category_Красота и здоровье _count': 'category_beauty_and_health_count',
    'category_Культура и искусство _count': 'category_culture_and_art_count',
    #'category_Кинотеатры_count': 'category_cinemas_count',
    'category_Магазины _count': 'category_stores_count',
    #'category_Торговые центры_count': 'category_shopping_centers_count',
    'category_Медицина и фармацевтика _count': 'category_medicine_and_pharmacy_count',
    'category_Больницы_count': 'category_hospitals_count',
    'category_Наука и образование _count': 'category_science_and_education_count',
    'category_Детские сады_count': 'category_kindergartens_count',
    'category_Вузы_count': 'category_universities_count',
    'category_Недвижимость _count': 'category_real_estate_count',
    'category_Бизнес-центры_count': 'category_business_centers_count',
    'category_Общежития_count': 'category_dormitories_count',
    'category_Общество _count': 'category_society_count',
    'category_Одежда и мода _count': 'category_clothing_and_fashion_count',
    #'category_Музеи_count': 'category_museums_count',
    'category_Отдых и развлечения _count': 'category_recreation_and_entertainment_count',
    'category_Зоопарки_count': 'category_zoos_count',
    #'category_Дельфинарии и океанариумы_count': 'category_dolphinariums_and_aquariums_count',
    'category_Развлекательные центры_count': 'category_entertainment_centers_count',
    'category_Производство и поставки _count': 'category_production_and_supplies_count',
    'category_Реклама и полиграфия _count': 'category_advertising_and_printing_count',
    'category_Справочно-информационные системы _count': 'category_reference_information_systems_count',
    'category_Средства массовой информации _count': 'category_mass_media_count',
    'category_Строительство и ремонт _count': 'category_construction_and_repair_count',
    'category_Телекоммуникации и связь _count': 'category_telecommunications_and_communication_count',
    'category_Товары для детей _count': 'category_goods_for_children_count',
    'category_Транспорт и перевозки _count': 'category_transport_and_transportation_count',
    'category_Железнодорожные вокзалы_count': 'category_railway_stations_count',
    'category_Автовокзалы_count': 'category_bus_stations_count',
    'category_Аэропорты_count': 'category_airports_count',
    'category_Гостиницы_count': 'category_hotels_count',
    'category_Туризм _count': 'category_tourism_count',
    'category_Услуги _count': 'category_services_count',
    'category_Финансы _count': 'category_finance_count',
    'category_Автостоянки и паркинги_count': 'category_parking_lots_and_garages_count',
    'category_Бесплатные парковки_count': 'category_free_parking_count',
    'category_Парковки_count': 'category_parking_count',
    'category_Платные парковки_count': 'category_paid_parking_count',
    'category_Подземные парковки_count': 'category_underground_parking_count',
    'category_Поликлиники_count': 'category_polyclinics_count',
    'category_Гимназии_count': 'category_gymnasiums_count',
    'category_Колледжи_count': 'category_colleges_count',
    'category_Лицеи_count': 'category_lyceums_count',
    'category_Общеобразовательные школы_count': 'category_general_education_schools_count',
    'category_Планетарии_count': 'category_planetariums_count',
    'category_Парки аттракционов_count': 'category_amusement_parks_count',
    #'category_Парки культуры и отдыха_count': 'category_parks_of_culture_and_recreation_count',
    'category_Производственные предприятия_count': 'category_industrial_enterprises_count'
}

df_yandex_categories.rename(columns=rename_dict, inplace=True)

### rebuid dataframes with addition of neighbours

In [454]:
df_base_yandex_categories = pd.merge(df_base_hes_full, df_yandex_categories,
                                how='left', left_on='h3_9', right_on='h3_9_yandex')
df_base_yandex_categories = df_base_yandex_categories.drop(['h3_9_yandex'], axis=1)
df_base_yandex_categories.shape

(7850, 53)

In [455]:
df_base_yandex_categories = df_base_yandex_categories.replace(np.nan, 0)

In [456]:
df_base_yandex_categories.head(3)

,h3_9,category_auto_count,category_security_count,category_business_count,category_government_count,category_pets_count,category_home_and_interior_count,category_food_and_drinks_count,category_computers_and_internet_count,category_beauty_and_health_count,category_culture_and_art_count,category_stores_count,category_medicine_and_pharmacy_count,category_hospitals_count,category_science_and_education_count,category_kindergartens_count,category_universities_count,category_real_estate_count,category_business_centers_count,category_dormitories_count,category_society_count,category_clothing_and_fashion_count,category_recreation_and_entertainment_count,category_zoos_count,category_entertainment_centers_count,category_production_and_supplies_count,category_advertising_and_printing_count,category_reference_information_systems_count,category_mass_media_count,category_construction_and_repair_count,category_telecommunications_and_communication_count,category_goods_for_children_count,category_transport_and_transportation_count,category_railway_stations_count,category_bus_stations_count,category_airports_count,category_hotels_count,category_tourism_count,category_services_count,category_finance_count,category_parking_lots_and_garages_count,category_free_parking_count,category_parking_count,category_paid_parking_count,category_underground_parking_count,category_polyclinics_count,category_gymnasiums_count,category_colleges_count,category_lyceums_count,category_general_education_schools_count,category_planetariums_count,category_amusement_parks_count,category_industrial_enterprises_count
0,8911060006bffff,32.0,3.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,89110600077ffff,31.0,7.0,5.0,0.0,0.0,0.0,11.0,5.0,0.0,0.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,8.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,89110600113ffff,38.0,4.0,18.0,0.0,0.0,9.0,54.0,5.0,64.0,11.0,20.0,6.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,25.0,0.0,3.0,1.0,7.0,0.0,0.0,13.0,5.0,14.0,5.0,0.0,0.0,0.0,0.0,6.0,13.0,9.0,6.0,1.0,6.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [457]:
def calculate_category_aggregates(df, neighbors_hexes):
    results = []

    # List of columns to aggregate
    columns_to_aggregate = df.columns[1:]  # all columns except 'h3_9'

    for hex_id in df['h3_9']:
        neighbors = h3.k_ring(hex_id, neighbors_hexes)
        
        # Get data for neighbors including the hex itself
        neighbors_data_including_hex = df[df['h3_9'].isin(neighbors)]
        
        # Dictionary to store the results for this hex_id
        result = {'h3_9': hex_id}
        
        # Calculations for each column
        for column in columns_to_aggregate:
            total_including_hex = neighbors_data_including_hex[column].sum()
            result[f'{column}_2_hn'] = total_including_hex
        
        results.append(result)
    
    return pd.DataFrame(results)

In [458]:
%%time
neighbors_hexes = 2
df_base_yandex_categories_neighbors = calculate_category_aggregates(df_base_yandex_categories, neighbors_hexes)

CPU times: total: 1min
Wall time: 1min 1s


In [459]:
df_base_yandex_categories_neighbors.shape

(7850, 53)

In [460]:
df_base_yandex_categories_neighbors.head(5)

,h3_9,category_auto_count_2_hn,category_security_count_2_hn,category_business_count_2_hn,category_government_count_2_hn,category_pets_count_2_hn,category_home_and_interior_count_2_hn,category_food_and_drinks_count_2_hn,category_computers_and_internet_count_2_hn,category_beauty_and_health_count_2_hn,category_culture_and_art_count_2_hn,category_stores_count_2_hn,category_medicine_and_pharmacy_count_2_hn,category_hospitals_count_2_hn,category_science_and_education_count_2_hn,category_kindergartens_count_2_hn,category_universities_count_2_hn,category_real_estate_count_2_hn,category_business_centers_count_2_hn,category_dormitories_count_2_hn,category_society_count_2_hn,category_clothing_and_fashion_count_2_hn,category_recreation_and_entertainment_count_2_hn,category_zoos_count_2_hn,category_entertainment_centers_count_2_hn,category_production_and_supplies_count_2_hn,category_advertising_and_printing_count_2_hn,category_reference_information_systems_count_2_hn,category_mass_media_count_2_hn,category_construction_and_repair_count_2_hn,category_telecommunications_and_communication_count_2_hn,category_goods_for_children_count_2_hn,category_transport_and_transportation_count_2_hn,category_railway_stations_count_2_hn,category_bus_stations_count_2_hn,category_airports_count_2_hn,category_hotels_count_2_hn,category_tourism_count_2_hn,category_services_count_2_hn,category_finance_count_2_hn,category_parking_lots_and_garages_count_2_hn,category_free_parking_count_2_hn,category_parking_count_2_hn,category_paid_parking_count_2_hn,category_underground_parking_count_2_hn,category_polyclinics_count_2_hn,category_gymnasiums_count_2_hn,category_colleges_count_2_hn,category_lyceums_count_2_hn,category_general_education_schools_count_2_hn,category_planetariums_count_2_hn,category_amusement_parks_count_2_hn,category_industrial_enterprises_count_2_hn
0,8911060006bffff,109.0,15.0,11.0,2.0,11.0,17.0,43.0,8.0,0.0,1.0,30.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,4.0,0.0,2.0,18.0,1.0,0.0,0.0,132.0,1.0,19.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,13.0,6.0,2.0,6.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,89110600077ffff,91.0,12.0,7.0,1.0,2.0,0.0,32.0,12.0,0.0,1.0,10.0,1.0,0.0,0.0,0.0,0.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,23.0,4.0,0.0,4.0,0.0,0.0,2.0,0.0,2.0,4.0,22.0,5.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,89110600113ffff,239.0,41.0,53.0,0.0,3.0,31.0,156.0,40.0,95.0,17.0,64.0,21.0,2.0,5.0,0.0,0.0,11.0,11.0,0.0,6.0,7.0,28.0,0.0,3.0,15.0,16.0,1.0,0.0,93.0,26.0,27.0,46.0,4.0,0.0,6.0,6.0,10.0,33.0,129.0,22.0,15.0,22.0,14.0,14.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
3,89110600127ffff,33.0,2.0,11.0,0.0,0.0,7.0,96.0,11.0,81.0,0.0,22.0,20.0,0.0,8.0,2.0,0.0,1.0,1.0,0.0,4.0,1.0,1.0,0.0,0.0,3.0,20.0,0.0,0.0,11.0,5.0,10.0,22.0,0.0,0.0,0.0,8.0,1.0,6.0,2.0,5.0,4.0,5.0,4.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,89110600183ffff,298.0,58.0,54.0,0.0,3.0,31.0,158.0,43.0,94.0,18.0,82.0,20.0,2.0,4.0,0.0,0.0,16.0,11.0,1.0,6.0,7.0,29.0,0.0,3.0,24.0,16.0,1.0,0.0,110.0,28.0,27.0,108.0,4.0,0.0,16.0,5.0,15.0,42.0,126.0,27.0,20.0,27.0,18.0,18.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


In [ ]:
#df_yandex_categories
#df_base_yandex_categories_neighbors

## agregate hexagons: subcategories**

In [23]:
filtered_yandex_subcategories = filtered_yandex_subcategories[['h3_9_yandex', 'sub_category']]

In [24]:
filtered_yandex_subcategories.sub_category.nunique()

29

In [25]:
filtered_yandex_subcategories.shape

(33125, 2)

In [26]:
filtered_yandex_subcategories.head(3)

,h3_9_yandex,sub_category
132662,89110602a47ffff,Кинотеатры
132663,89110600813ffff,Кинотеатры
132664,89110600887ffff,Кинотеатры


In [27]:
%%time
unique_objects = filtered_yandex_subcategories['sub_category'].unique()
df_yandex_sub_categories = pd.DataFrame(index=df_yandex['h3_9_yandex'].unique())

for obj in unique_objects:
    df_yandex_sub_categories[f'{obj}_count'] = filtered_yandex_subcategories[filtered_yandex_subcategories['sub_category'] == obj].groupby('h3_9_yandex')['sub_category'].count()

df_yandex_sub_categories = df_yandex_sub_categories.fillna(0).astype(int)
df_yandex_sub_categories.insert(0, 'h3_9_yandex', df_yandex_sub_categories.index)
df_yandex_sub_categories = df_yandex_sub_categories.reset_index(drop=True)

CPU times: total: 250 ms
Wall time: 277 ms


In [28]:
for column in df_yandex_sub_categories.columns:
    if column not in ['h3_9_yandex']:
        df_yandex_sub_categories.rename(columns={column: 'subcategory_' + column}, inplace=True)

In [29]:
df_yandex_sub_categories.shape

(7755, 30)

In [30]:
df_yandex_sub_categories.head(3)

,h3_9_yandex,subcategory_Кинотеатры_count,subcategory_Торговые центры_count,subcategory_Больницы_count,subcategory_Детские сады_count,subcategory_Вузы_count,subcategory_Бизнес-центры_count,subcategory_Общежития_count,subcategory_Музеи_count,subcategory_Зоопарки_count,subcategory_Дельфинарии и океанариумы_count,subcategory_Развлекательные центры_count,subcategory_Железнодорожные вокзалы_count,subcategory_Автовокзалы_count,subcategory_Аэропорты_count,subcategory_Гостиницы_count,subcategory_Автостоянки и паркинги_count,subcategory_Бесплатные парковки_count,subcategory_Парковки_count,subcategory_Платные парковки_count,subcategory_Подземные парковки_count,subcategory_Поликлиники_count,subcategory_Гимназии_count,subcategory_Колледжи_count,subcategory_Лицеи_count,subcategory_Общеобразовательные школы_count,subcategory_Планетарии_count,subcategory_Парки аттракционов_count,subcategory_Парки культуры и отдыха_count,subcategory_Производственные предприятия_count
0,89110610b7bffff,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,2
1,89110610b97ffff,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,3,0,2,0,0,0,0,0,0,0,0,0,0,0
2,89110610b13ffff,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
#df_yandex_categories
#df_yandex_sub_categories

**neghbors for unique_attraction_point**

In [32]:
transport = ['metro_count', 'total_metro_neighbour_and_hex']
unique_attraction_points = ['subcategory_Кинотеатры_count', 'subcategory_Торговые центры_count', 'subcategory_Музеи_count',
                            'subcategory_Парки культуры и отдыха_count']

hex_unique_attraction_points = ['h3_9_yandex','subcategory_Кинотеатры_count', 'subcategory_Торговые центры_count', 'subcategory_Музеи_count',
                            'subcategory_Парки культуры и отдыха_count']

work_study_related_objects = [ 'subcategory_Вузы_count', 'subcategory_Бизнес-центры_count', 'subcategory_Общежития_count',
                               'subcategory_Колледжи_count', 'subcategory_Производственные предприятия_count']

stops_auto_and_bus = ['subcategory_Автостоянки и паркинги_count'] #add bus stops

In [33]:
df_yandex_sub_categories.head(3)

,h3_9_yandex,subcategory_Кинотеатры_count,subcategory_Торговые центры_count,subcategory_Больницы_count,subcategory_Детские сады_count,subcategory_Вузы_count,subcategory_Бизнес-центры_count,subcategory_Общежития_count,subcategory_Музеи_count,subcategory_Зоопарки_count,subcategory_Дельфинарии и океанариумы_count,subcategory_Развлекательные центры_count,subcategory_Железнодорожные вокзалы_count,subcategory_Автовокзалы_count,subcategory_Аэропорты_count,subcategory_Гостиницы_count,subcategory_Автостоянки и паркинги_count,subcategory_Бесплатные парковки_count,subcategory_Парковки_count,subcategory_Платные парковки_count,subcategory_Подземные парковки_count,subcategory_Поликлиники_count,subcategory_Гимназии_count,subcategory_Колледжи_count,subcategory_Лицеи_count,subcategory_Общеобразовательные школы_count,subcategory_Планетарии_count,subcategory_Парки аттракционов_count,subcategory_Парки культуры и отдыха_count,subcategory_Производственные предприятия_count
0,89110610b7bffff,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,2
1,89110610b97ffff,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,3,0,2,0,0,0,0,0,0,0,0,0,0,0
2,89110610b13ffff,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
df_yandex_unique_points = df_yandex_sub_categories[hex_unique_attraction_points]

**drop unique columns since we will rejoin them with neghbors aggregation**

In [35]:
df_yandex_sub_categories = df_yandex_sub_categories.drop(unique_attraction_points, axis=1)

In [36]:
df_yandex_sub_categories.shape

(7755, 26)

**continue**

In [37]:
df_yandex_unique_points.shape

(7755, 5)

In [38]:
df_yandex_unique_points = df_yandex_unique_points.rename(columns={
    'h3_9_yandex': 'h3_9_yandex',
    'subcategory_Кинотеатры_count': 'subcategory_cinemas_count',
    'subcategory_Торговые центры_count': 'subcategory_shopping_centers_count',
    'subcategory_Музеи_count': 'subcategory_museums_count',
    'subcategory_Парки культуры и отдыха_count': 'subcategory_parks_of_culture_and_leisure_count'
})

In [39]:
df_yandex_unique_points.head(5)

,h3_9_yandex,subcategory_cinemas_count,subcategory_shopping_centers_count,subcategory_museums_count,subcategory_parks_of_culture_and_leisure_count
0,89110610b7bffff,0,0,0,0
1,89110610b97ffff,0,0,0,0
2,89110610b13ffff,0,0,0,0
3,89110610b2bffff,0,0,0,0
4,89110610c23ffff,0,0,0,0


In [40]:
df_yandex_unique_points.describe().round(2)

,subcategory_cinemas_count,subcategory_shopping_centers_count,subcategory_museums_count,subcategory_parks_of_culture_and_leisure_count
count,7755.00,7755.00,7755.00,7755.00
mean,0.07,0.21,0.07,0.05
std,0.40,0.80,0.46,0.28
min,0.00,0.00,0.00,0.00
25%,0.00,0.00,0.00,0.00
50%,0.00,0.00,0.00,0.00
75%,0.00,0.00,0.00,0.00
max,6.00,15.00,11.00,4.00


In [41]:
import h3
from scipy import stats

def calculate_yandex_aggregates_for_all(df, neighbors_hexes, h3_9_name='h3_9_yandex'):
    results = []
    
    # List of columns to aggregate
    columns_to_aggregate = [
        'subcategory_cinemas_count',
        'subcategory_shopping_centers_count',
        'subcategory_museums_count',
        'subcategory_parks_of_culture_and_leisure_count'
    ]
    
    for hex_id in df[h3_9_name]:
        neighbors = h3.k_ring(hex_id, neighbors_hexes)
        
        # Get data for neighbors including the hex itself
        neighbors_data_including_hex = df[df[h3_9_name].isin(neighbors)]
        
        # Exclude the hex itself for neighbors only
        neighbors.remove(hex_id)
        neighbors_data_excluding_hex = df[df[h3_9_name].isin(neighbors)]
        
        # Dictionary to store the results for this hex_id
        result = {'hex_id': hex_id, 'neighbor_count': len(neighbors)}
        
        # Calculations for each column
        for column in columns_to_aggregate:
            total_including_hex = neighbors_data_including_hex[column].sum()
            total_excluding_hex = neighbors_data_excluding_hex[column].sum()
            result[f'total_{column}_including_hex'] = total_including_hex
            result[f'total_{column}_excluding_hex'] = total_excluding_hex
        
        results.append(result)
    
    return pd.DataFrame(results)


In [42]:
%%time
neighbors_hexes = 4
df_yandex_unique_points_aggregated = calculate_yandex_aggregates_for_all(df_yandex_unique_points, neighbors_hexes)

CPU times: total: 23.6 s
Wall time: 23.9 s


In [43]:
df_yandex_unique_points_aggregated.shape

(7755, 10)

In [44]:
df_yandex_unique_points_aggregated = df_yandex_unique_points_aggregated[['hex_id', 'total_subcategory_cinemas_count_including_hex',
                                                                        'total_subcategory_shopping_centers_count_including_hex',
                                                                        'total_subcategory_museums_count_including_hex',
                                                                        'total_subcategory_parks_of_culture_and_leisure_count_including_hex']]

In [45]:
df_yandex_unique_points_aggregated.head(3)

,hex_id,total_subcategory_cinemas_count_including_hex,total_subcategory_shopping_centers_count_including_hex,total_subcategory_museums_count_including_hex,total_subcategory_parks_of_culture_and_leisure_count_including_hex
0,89110610b7bffff,0,0,1,0
1,89110610b97ffff,0,7,0,1
2,89110610b13ffff,0,2,0,0


In [46]:
df_yandex_unique_points_aggregated.describe().round(2)

,total_subcategory_cinemas_count_including_hex,total_subcategory_shopping_centers_count_including_hex,total_subcategory_museums_count_including_hex,total_subcategory_parks_of_culture_and_leisure_count_including_hex
count,7755.00,7755.00,7755.00,7755.00
mean,4.09,11.78,4.10,2.82
std,8.53,15.42,12.61,4.33
min,0.00,0.00,0.00,0.00
25%,0.00,0.00,0.00,0.00
50%,1.00,6.00,1.00,1.00
75%,5.00,18.00,3.00,4.00
max,90.00,108.00,135.00,29.00


**переделываем колонки в бинарные, если больше 0 - 1, равно 0 - 0**

In [47]:
def create_binary_dataframe(df, columns_to_convert, id_column):
    binary_df = df.copy()
    
    for column in columns_to_convert:
        binary_df[column] = binary_df[column].apply(lambda x: 1 if x > 1 else 0)
    columns_to_keep = [id_column]
    
    final_df = pd.concat([binary_df[columns_to_convert], df[columns_to_keep]], axis=1)
    
    return final_df

In [48]:
df_yandex_unique_points_aggregated.columns

Index(['hex_id', 'total_subcategory_cinemas_count_including_hex',
       'total_subcategory_shopping_centers_count_including_hex',
       'total_subcategory_museums_count_including_hex',
       'total_subcategory_parks_of_culture_and_leisure_count_including_hex'],
      dtype='object')

In [49]:
columns_to_convert = ['total_subcategory_cinemas_count_including_hex',
       'total_subcategory_shopping_centers_count_including_hex',
       'total_subcategory_museums_count_including_hex',
       'total_subcategory_parks_of_culture_and_leisure_count_including_hex']

In [367]:
df_yandex_unique_points_aggregated_binary = create_binary_dataframe(df_yandex_unique_points_aggregated, columns_to_convert, 'hex_id')
df_yandex_unique_points_aggregated_binary.shape

(7755, 5)

In [368]:
df_yandex_unique_points_aggregated_binary.head(3)

,total_subcategory_cinemas_count_including_hex,total_subcategory_shopping_centers_count_including_hex,total_subcategory_museums_count_including_hex,total_subcategory_parks_of_culture_and_leisure_count_including_hex,hex_id
0,0,0,0,0,89110610b7bffff
1,0,1,0,0,89110610b97ffff
2,0,1,0,0,89110610b13ffff


**renaming columns of df_yandex_sub_categories**

In [135]:
rename_dict_subcategories = {
    'subcategory_Больницы_count': 'subcategory_hospitals_count',
    'subcategory_Детские сады_count': 'subcategory_kindergartens_count',
    'subcategory_Вузы_count': 'subcategory_universities_count',
    'subcategory_Бизнес-центры_count': 'subcategory_business_centers_count',
    'subcategory_Общежития_count': 'subcategory_dormitories_count',
    'subcategory_Зоопарки_count': 'subcategory_zoos_count',
    'subcategory_Дельфинарии и океанариумы_count': 'subcategory_dolphinariums_and_aquariums_count',
    'subcategory_Развлекательные центры_count': 'subcategory_entertainment_centers_count',
    'subcategory_Железнодорожные вокзалы_count': 'subcategory_railway_stations_count',
    'subcategory_Автовокзалы_count': 'subcategory_bus_stations_count',
    'subcategory_Аэропорты_count': 'subcategory_airports_count',
    'subcategory_Гостиницы_count': 'subcategory_hotels_count',
    'subcategory_Автостоянки и паркинги_count': 'subcategory_parking_lots_and_garages_count',
    'subcategory_Бесплатные парковки_count': 'subcategory_free_parking_count',
    'subcategory_Парковки_count': 'subcategory_parking_count',
    'subcategory_Платные парковки_count': 'subcategory_paid_parking_count',
    'subcategory_Подземные парковки_count': 'subcategory_underground_parking_count',
    'subcategory_Поликлиники_count': 'subcategory_polyclinics_count',
    'subcategory_Гимназии_count': 'subcategory_gymnasiums_count',
    'subcategory_Колледжи_count': 'subcategory_colleges_count',
    'subcategory_Лицеи_count': 'subcategory_lyceums_count',
    'subcategory_Общеобразовательные школы_count': 'subcategory_general_education_schools_count',
    'subcategory_Планетарии_count': 'subcategory_planetariums_count',
    'subcategory_Парки аттракционов_count': 'subcategory_amusement_parks_count',
    'subcategory_Производственные предприятия_count': 'subcategory_industrial_enterprises_count'
}

df_yandex_sub_categories.rename(columns=rename_dict_subcategories, inplace=True)

### rebuid dataframes with addition of neighbours

In [461]:
df_base_yandex_sub_categories = pd.merge(df_base_hes_full, df_yandex_sub_categories,
                                how='left', left_on='h3_9', right_on='h3_9_yandex')
df_base_yandex_sub_categories = df_base_yandex_sub_categories.drop(['h3_9_yandex'], axis=1)
df_base_yandex_sub_categories.shape

(7850, 26)

In [462]:
df_base_yandex_sub_categories = df_base_yandex_sub_categories.replace(np.nan, 0)

In [463]:
df_base_yandex_sub_categories.head(3)

,h3_9,subcategory_hospitals_count,subcategory_kindergartens_count,subcategory_universities_count,subcategory_business_centers_count,subcategory_dormitories_count,subcategory_zoos_count,subcategory_dolphinariums_and_aquariums_count,subcategory_entertainment_centers_count,subcategory_railway_stations_count,subcategory_bus_stations_count,subcategory_airports_count,subcategory_hotels_count,subcategory_parking_lots_and_garages_count,subcategory_free_parking_count,subcategory_parking_count,subcategory_paid_parking_count,subcategory_underground_parking_count,subcategory_polyclinics_count,subcategory_gymnasiums_count,subcategory_colleges_count,subcategory_lyceums_count,subcategory_general_education_schools_count,subcategory_planetariums_count,subcategory_amusement_parks_count,subcategory_industrial_enterprises_count
0,8911060006bffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,89110600077ffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,89110600113ffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,6.0,1.0,6.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [464]:
def calculate_category_aggregates(df, neighbors_hexes):
    results = []

    # List of columns to aggregate
    columns_to_aggregate = df.columns[1:]  # all columns except 'h3_9'

    for hex_id in df['h3_9']:
        neighbors = h3.k_ring(hex_id, neighbors_hexes)
        
        # Get data for neighbors including the hex itself
        neighbors_data_including_hex = df[df['h3_9'].isin(neighbors)]
        
        # Dictionary to store the results for this hex_id
        result = {'h3_9': hex_id}
        
        # Calculations for each column
        for column in columns_to_aggregate:
            total_including_hex = neighbors_data_including_hex[column].sum()
            result[f'{column}_2_hn'] = total_including_hex
        
        results.append(result)
    
    return pd.DataFrame(results)

In [465]:
%%time
neighbors_hexes = 2
df_base_yandex_sub_categories_neighbors = calculate_category_aggregates(df_base_yandex_sub_categories, neighbors_hexes)

CPU times: total: 33 s
Wall time: 34.2 s


In [466]:
df_base_yandex_sub_categories_neighbors.shape

(7850, 26)

In [467]:
df_base_yandex_sub_categories_neighbors.head(5)

,h3_9,subcategory_hospitals_count_2_hn,subcategory_kindergartens_count_2_hn,subcategory_universities_count_2_hn,subcategory_business_centers_count_2_hn,subcategory_dormitories_count_2_hn,subcategory_zoos_count_2_hn,subcategory_dolphinariums_and_aquariums_count_2_hn,subcategory_entertainment_centers_count_2_hn,subcategory_railway_stations_count_2_hn,subcategory_bus_stations_count_2_hn,subcategory_airports_count_2_hn,subcategory_hotels_count_2_hn,subcategory_parking_lots_and_garages_count_2_hn,subcategory_free_parking_count_2_hn,subcategory_parking_count_2_hn,subcategory_paid_parking_count_2_hn,subcategory_underground_parking_count_2_hn,subcategory_polyclinics_count_2_hn,subcategory_gymnasiums_count_2_hn,subcategory_colleges_count_2_hn,subcategory_lyceums_count_2_hn,subcategory_general_education_schools_count_2_hn,subcategory_planetariums_count_2_hn,subcategory_amusement_parks_count_2_hn,subcategory_industrial_enterprises_count_2_hn
0,8911060006bffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,6.0,2.0,6.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,89110600077ffff,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,89110600113ffff,2.0,0.0,0.0,11.0,0.0,0.0,0.0,3.0,4.0,0.0,6.0,6.0,22.0,15.0,22.0,14.0,14.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
3,89110600127ffff,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,5.0,4.0,5.0,4.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,89110600183ffff,2.0,0.0,0.0,11.0,1.0,0.0,0.0,3.0,4.0,0.0,16.0,5.0,27.0,20.0,27.0,18.0,18.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


In [468]:
#df_yandex_unique_points_aggregated_binary

#df_base_yandex_sub_categories
#df_base_yandex_sub_categories_neighbors

#df_yandex_categories
#df_base_yandex_categories_neighbors

## Population

In [469]:
%%time
df_population = pd.read_excel('C:/Users/elysi/Tele2_project/data/geography/population_Sanya.xlsx')

CPU times: total: 5.06 s
Wall time: 5.22 s


In [470]:
df_population = df_population[['h3_9_population', 'Население']]

In [471]:
df_population.shape

(22617, 2)

In [472]:
df_population.h3_9_population.nunique()

3650

In [473]:
df_population.head(3)

,h3_9_population,Население
0,89110606407ffff,NaN
1,89110614a07ffff,66.0
2,89110614a07ffff,81.0


In [474]:
df_population_aggregated = df_population.groupby('h3_9_population').agg({'Население': 'sum'}).reset_index()
df_population_aggregated.rename(columns={'Население': 'population_sum'}, inplace=True)

In [475]:
df_population_aggregated.shape

(3650, 2)

In [476]:
df_population_aggregated.head(3)

,h3_9_population,population_sum
0,8911044121bffff,0.0
1,89110441243ffff,0.0
2,89110441447ffff,9.0


In [477]:
df_population_aggregated.population_sum.value_counts(normalize = True)

0.0       0.175068
6.0       0.009863
12.0      0.006849
72.0      0.005753
24.0      0.005753
            ...   
3462.0    0.000274
2556.0    0.000274
4587.0    0.000274
4152.0    0.000274
1338.0    0.000274
Name: population_sum, Length: 1344, dtype: float64

**function for count of object for neighbors**

In [478]:
new_h3_9_values = df_yandex_categories[~df_yandex_categories['h3_9_yandex'].isin(df_population_aggregated['h3_9_population'])][['h3_9_yandex']]

# Create a dataframe with these new values and metro_count as 0
new_entries = new_h3_9_values.rename(columns={'h3_9_yandex': 'h3_9_population'})
new_entries['population_sum'] = 0

# Append the new entries to df_metro_agr
df_population_agr_updated = pd.concat([df_population_aggregated, new_entries], ignore_index=True)

In [479]:
df_population_agr_updated.shape

(8509, 2)

In [480]:
df_population_agr_updated.population_sum.isnull().sum()

0

In [481]:
def calculate_population_aggregates(df, neighbors_hexes, h3_9_name='h3_9_population', population_sum_name='population_sum'):
    results = []
    
    for hex_id in df[h3_9_name]:
        neighbors = h3.k_ring(hex_id, neighbors_hexes)
        
        # Get data for neighbors including the hex itself
        neighbors_data_including_hex = df[df[h3_9_name].isin(neighbors)]
        
        # Exclude the hex itself for neighbors only
        neighbors.remove(hex_id)
        neighbors_data_excluding_hex = df[df[h3_9_name].isin(neighbors)]
        
        # Calculations
        total_population_including_hex = neighbors_data_including_hex[population_sum_name].sum()
        total_population_excluding_hex = neighbors_data_excluding_hex[population_sum_name].sum()

        results.append({
            'hex_id': hex_id,
            'total_population_excluding_hex': total_population_excluding_hex,
            'total_population_neighbour_2_hn': total_population_including_hex
        })
    
    return pd.DataFrame(results)

In [482]:
# Example usage with your data
neighbors_hexes = 2  # specify the number of neighbor hexagons
df_population_agr_neighbors = calculate_population_aggregates(df_population_agr_updated, neighbors_hexes)

In [483]:
df_population_agr_neighbors.shape

(8509, 3)

In [484]:
df_population_agr_neighbors = df_population_agr_neighbors[['hex_id', 'total_population_neighbour_2_hn']]

In [485]:
df_population_agr_neighbors.head(3)

,hex_id,total_population_neighbour_2_hn
0,8911044121bffff,0.0
1,89110441243ffff,0.0
2,89110441447ffff,9.0


In [486]:
df_population_agr_updated = pd.merge(df_population_agr_updated, df_population_agr_neighbors,
                                how='left', left_on='h3_9_population', right_on='hex_id')

df_population_agr_updated = df_population_agr_updated.drop(['hex_id'], axis=1)

In [487]:
df_population_agr_updated.shape

(8509, 3)

In [488]:
df_population_agr_updated.head(3)

,h3_9_population,population_sum,total_population_neighbour_2_hn
0,8911044121bffff,0.0,0.0
1,89110441243ffff,0.0,0.0
2,89110441447ffff,9.0,9.0


In [489]:
#df_yandex_unique_points_aggregated_binary

#df_yandex_categories
#df_yandex_sub_categories
#df_population_agr_updated

## Metro

In [490]:
%%time
df_metro = pd.read_excel('C:/Users/elysi/Tele2_project/data/geography/metro_objects_Sanya.xlsx')

CPU times: total: 31.2 ms
Wall time: 44 ms


In [491]:
df_metro.shape

(71, 7)

In [492]:
df_metro['Наименование'].nunique()

71

In [493]:
df_metro.h3_9_metro.nunique()

67

In [494]:
df_metro.head(3)

,№,Наименование,Адрес,Координаты,lat,lon,h3_9_metro
0,1,Проспект Ветеранов,"г.Санкт-Петербург, бульвар Новаторов, соор.2, ...","59.853406, 30.269578",59.853406,30.269578,89110602d53ffff
1,3,Ленинский проспект,"г.Санкт-Петербург, бульвар Новаторов, соор.1, ...","59.853514, 30.267188",59.853514,30.267188,89110602dcbffff
2,5,Автово,"г.Санкт-Петербург, пр. Стачек, д.90, к.2, лит.А","59.867290, 30.261277",59.867290,30.261277,89110615e4bffff


In [495]:
df_metro_agr = df_metro.groupby('h3_9_metro').agg(
    metro_count=('Наименование', 'nunique')
).reset_index()

In [496]:
df_metro_agr.shape

(67, 2)

In [497]:
df_metro_agr.head(5)

,h3_9_metro,metro_count
0,8911060088fffff,1
1,89110600947ffff,1
2,89110600b4fffff,1
3,89110600cafffff,1
4,89110602923ffff,1


**add all hexagons to df, to determine neihgbours_objects**

In [498]:
# Find h3_9_yandex values that do not exist in df_metro_agr
new_h3_9_values = df_yandex_categories[~df_yandex_categories['h3_9_yandex'].isin(df_metro_agr['h3_9_metro'])][['h3_9_yandex']]

# Create a dataframe with these new values and metro_count as 0
new_entries = new_h3_9_values.rename(columns={'h3_9_yandex': 'h3_9_metro'})
new_entries['metro_count'] = 0

# Append the new entries to df_metro_agr
df_metro_agr_updated = pd.concat([df_metro_agr, new_entries], ignore_index=True)

In [499]:
df_metro_agr_updated.shape

(7755, 2)

In [500]:
df_metro_agr_updated.metro_count.value_counts()

0    7688
1      64
2       2
3       1
Name: metro_count, dtype: int64

In [501]:
df_metro_agr_updated.head(3)

,h3_9_metro,metro_count
0,8911060088fffff,1
1,89110600947ffff,1
2,89110600b4fffff,1


**function for count of object for neighbors**

In [502]:
import h3
def calculate_fit_club_sums(hex_id, df, neighbors_hexes, h3_9_name, total_name):
    neighbors = h3.k_ring(hex_id, neighbors_hexes)
    # Including the hex itself in neighbors for the total_fit_clubs_neighbour_and_hex calculation
    total_neighbors = df[df[h3_9_name].isin(neighbors)][total_name].sum()
    # Exclude the hex itself for the total_fit_clubs_neighbour calculation
    neighbors.remove(hex_id)
    total_neighbors_excluding_hex = df[df[h3_9_name].isin(neighbors)][total_name].sum()
    return len(neighbors), total_neighbors_excluding_hex, total_neighbors

In [503]:
df_metro_agr_neghbors = df_metro_agr_updated.copy()

In [504]:
df_metro_agr_neghbors[['neighbour_amount', 'total_metro_only_neighbour_5_hn', 'total_metro_neighbour_5_hn']] = \
    df_metro_agr_neghbors.apply(lambda row: calculate_fit_club_sums(row['h3_9_metro'], df_metro_agr_neghbors,
                                                                    neighbors_hexes = 4, h3_9_name = 'h3_9_metro',
                                                                    total_name = 'metro_count'),
                                axis=1, result_type='expand')

In [505]:
df_metro_agr_neghbors.shape

(7755, 5)

In [506]:
df_metro_agr_neghbors.head(3)

,h3_9_metro,metro_count,neighbour_amount,total_metro_only_neighbour_5_hn,total_metro_neighbour_5_hn
0,8911060088fffff,1,60,0,1
1,89110600947ffff,1,60,0,1
2,89110600b4fffff,1,60,0,1


In [507]:
df_metro_agr_neghbors.neighbour_amount.value_counts()

60    7755
Name: neighbour_amount, dtype: int64

In [508]:
df_metro_agr_neghbors.total_metro_neighbour_5_hn.value_counts()

0     5237
1     1681
2      593
3       87
4       39
5       30
7       29
6       28
8       13
10       7
9        6
11       4
12       1
Name: total_metro_neighbour_5_hn, dtype: int64

In [ ]:
#df_metro_agr_neghbors
#df_yandex_unique_points_aggregated_binary

#df_yandex_categories
#df_yandex_sub_categories
#df_population_agr_updated

## Cian

In [ ]:
%%time
df_cian = pd.read_excel('C:/Users/elysi/Tele2_project/data/geography/Cian_Sanya_aggregation.xlsx')

In [ ]:
df_cian.Гексагоны.nunique()

In [ ]:
df_cian_cleaned = df_cian[df_cian['Гексагоны'].isin(df_yandex['h3_9_yandex'])]
df_cian_cleaned.shape

In [ ]:
#df_cian_cleaned = df_cian.dropna(how='all', subset=[col for col in df_cian.columns if col != 'Гексагоны'])

In [ ]:
df_cian_cleaned.Гексагоны.nunique()

In [ ]:
df_cian_cleaned = df_cian_cleaned.round(2)

In [ ]:
df_cian_cleaned.head(3)

In [ ]:
for column in df_cian_cleaned.columns:
    if column not in ['Гексагоны']:
        df_cian_cleaned.rename(columns={column: 'cian_' + column}, inplace=True)

In [ ]:
df_cian_cleaned = df_cian_cleaned.rename(columns={
    'Гексагоны': 'h3_9_yandex',
    'cian_Количество объявлений': 'cian_listing_count',
    'cian_Общая площадь': 'cian_total_area',
    'cian_Средняя площадь': 'cian_avg_area',
    'cian_Мода площадь': 'cian_mode_area',
    'cian_Средняя цена': 'cian_avg_price',
    'cian_Средняя цена/м2': 'cian_avg_price_per_m2',
    'cian_Мода цена/м2': 'cian_mode_price_per_m2'
})


In [ ]:
df_cian_cleaned.head(5)

In [ ]:
import h3
from scipy import stats

def calculate_cian_aggregates_for_all(df, neighbors_hexes, h3_9_name='h3_9_yandex', listing_count_name='cian_listing_count', total_area_name='cian_total_area', avg_price_per_m2_name='cian_avg_price_per_m2', median_area_name='cian_mode_area', median_price_per_m2_name='cian_mode_price_per_m2'):
    results = []
    
    for hex_id in df[h3_9_name]:
        neighbors = h3.k_ring(hex_id, neighbors_hexes)
        
        # Get data for neighbors including the hex itself
        neighbors_data_including_hex = df[df[h3_9_name].isin(neighbors)]
        
        # Exclude the hex itself for neighbors only
        neighbors.remove(hex_id)
        neighbors_data_excluding_hex = df[df[h3_9_name].isin(neighbors)]
        
        # Calculations including the hex itself
        total_listing_count_including_hex = neighbors_data_including_hex[listing_count_name].sum()
        total_area_including_hex = neighbors_data_including_hex[total_area_name].sum()
        median_area_including_hex = neighbors_data_including_hex[median_area_name].median()
        avg_area_including_hex = neighbors_data_including_hex[median_area_name].mean()
        median_price_per_m2_including_hex = neighbors_data_including_hex[median_price_per_m2_name].median()
        avg_price_per_m2_including_hex = neighbors_data_including_hex[avg_price_per_m2_name].mean()

        # Calculations excluding the hex itself
        total_listing_count_excluding_hex = neighbors_data_excluding_hex[listing_count_name].sum()
        total_area_excluding_hex = neighbors_data_excluding_hex[total_area_name].sum()
        median_area_excluding_hex = neighbors_data_excluding_hex[median_area_name].median()
        avg_area_excluding_hex = neighbors_data_excluding_hex[median_area_name].mean()
        median_price_per_m2_excluding_hex = neighbors_data_excluding_hex[median_price_per_m2_name].median()
        avg_price_per_m2_excluding_hex = neighbors_data_excluding_hex[avg_price_per_m2_name].mean()

        results.append({
            'h3_9_yandex': hex_id,
            'neighbor_count': len(neighbors),
            'total_listing_count_excluding_hex': total_listing_count_excluding_hex,
            'total_listing_count_including_hex': total_listing_count_including_hex,
            'total_area_excluding_hex': total_area_excluding_hex,
            'total_area_including_hex': total_area_including_hex,
            'median_area_excluding_hex': median_area_excluding_hex,
            'median_area_including_hex': median_area_including_hex,
            'avg_area_excluding_hex': avg_area_excluding_hex,
            'avg_area_including_hex': avg_area_including_hex,
            'median_price_per_m2_excluding_hex': median_price_per_m2_excluding_hex,
            'median_price_per_m2_including_hex': median_price_per_m2_including_hex,
            'avg_price_per_m2_excluding_hex': avg_price_per_m2_excluding_hex,
            'avg_price_per_m2_including_hex': avg_price_per_m2_including_hex
        })
    
    return pd.DataFrame(results)

In [ ]:
%%time
neighbors_hexes = 3
df_cian_aggregated = calculate_cian_aggregates_for_all(df_cian_cleaned, neighbors_hexes)

In [ ]:
df_cian_aggregated.head(5)

In [ ]:
df_cian_aggregated.h3_9_yandex.nunique()

In [ ]:
df_cian_aggregated.median_price_per_m2_including_hex.isnull().sum()

In [ ]:
df_cian_aggregated.median_price_per_m2_including_hex.value_counts()

**совмещаем датасет с соседями и без**

In [ ]:
df_cian_aggregated = df_cian_aggregated[['h3_9_yandex', 'total_listing_count_including_hex', 'total_area_including_hex',
                                         'median_area_including_hex', 'avg_area_including_hex',
                                         'median_price_per_m2_including_hex', 'avg_price_per_m2_including_hex']]

In [ ]:
for column in df_cian_aggregated.columns:
    if column not in ['h3_9_yandex']:
        df_cian_aggregated.rename(columns={column: 'cian_' + column}, inplace=True)

In [ ]:
for column in df_cian_aggregated.columns:
    if column not in ['h3_9_yandex']:
        df_cian_aggregated.rename(columns={column: column + '_3hn'}, inplace=True)

In [ ]:
df_cian_aggregated.head(5)

In [410]:
#df_cian_aggregated
#df_metro_agr_neghbors
#df_population_agr_updated

#df_yandex_unique_points_aggregated_binary

#df_base_yandex_sub_categories
#df_base_yandex_sub_categories_neighbors

#df_yandex_categories
#df_base_yandex_categories_neighbors

## Fitness neibgours

**we decided to keep fitnees clubs from 2Gis (DELETE all sport related columns from yandex df**

In [379]:
df_geo = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_geo_features_spb.csv')

In [380]:
df_geo.total_fit_clubs.sum()

582.0

In [381]:
df_geo[['h3_9', 'total_fit_clubs']]

,h3_9,total_fit_clubs
0,8911060006bffff,0.0
1,89110600077ffff,0.0
2,89110600113ffff,4.0
3,89110600127ffff,0.0
4,89110600183ffff,0.0
...,...,...
3900,891106b9e4fffff,0.0
3901,891106b9e6bffff,0.0
3902,891106b9e6fffff,0.0
3903,891106b9e77ffff,0.0


In [382]:
import h3
def calculate_fit_club_sums(hex_id, df):
    neighbors = h3.k_ring(hex_id, 2)
    # Including the hex itself in neighbors for the total_fit_clubs_neighbour_and_hex calculation
    total_neighbors = df[df['h3_9'].isin(neighbors)]['total_fit_clubs'].sum()
    # Exclude the hex itself for the total_fit_clubs_neighbour calculation
    neighbors.remove(hex_id)
    total_neighbors_excluding_hex = df[df['h3_9'].isin(neighbors)]['total_fit_clubs'].sum()
    return len(neighbors), total_neighbors_excluding_hex, total_neighbors

In [383]:
df_data_lens = df_geo[['h3_9', 'total_fit_clubs']].copy()

In [384]:
df_data_lens[['neighbour_amount', 'total_fit_clubs_only_neighbour', 'total_fit_clubs_neighbour_2_hn']] = \
    df_data_lens.apply(lambda row: calculate_fit_club_sums(row['h3_9'], df_data_lens), axis=1, result_type='expand')

In [385]:
df_data_lens.shape

(3905, 5)

In [386]:
df_data_lens.head(3)

,h3_9,total_fit_clubs,neighbour_amount,total_fit_clubs_only_neighbour,total_fit_clubs_neighbour_2_hn
0,8911060006bffff,0.0,18.0,0.0,0.0
1,89110600077ffff,0.0,18.0,0.0,0.0
2,89110600113ffff,4.0,18.0,0.0,4.0


In [388]:
df_sport_final = df_data_lens[['h3_9', 'total_fit_clubs', 'total_fit_clubs_neighbour_2_hn']]

In [389]:
df_sport_final.head(3)

,h3_9,total_fit_clubs,total_fit_clubs_neighbour_2_hn
0,8911060006bffff,0.0,0.0
1,89110600077ffff,0.0,0.0
2,89110600113ffff,4.0,4.0


## Join of all data

In [ ]:
#df_sport_final
#df_cian_aggregated
#df_metro_agr_neghbors
#df_population_agr_updated
#df_yandex_unique_points_aggregated_binary

#df_base_yandex_sub_categories
#df_base_yandex_sub_categories_neighbors

#df_yandex_categories
#df_base_yandex_categories_neighbors

In [541]:
df_metro_agr_neghbors = df_metro_agr_neghbors[['h3_9_metro', 'metro_count', 'total_metro_neighbour_5_hn']]

In [542]:
df_population_agr_updated.head(3)

,h3_9_population,population_sum,total_population_neighbour_2_hn
0,8911044121bffff,0.0,0.0
1,89110441243ffff,0.0,0.0
2,89110441447ffff,9.0,9.0


**basic SPB hexagons**

In [543]:
df_time = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_time_based_features_full.csv')

In [544]:
df_geo_spb = df_time[['h3_9']]

In [545]:
df_geo_spb.shape

(3892, 1)

**transport and adm districts**

In [546]:
%%time
df_transport_agr = pd.read_csv('C:/Users/elysi/Tele2_project/data/geography/df_transport_agr.csv')
df_adm_distr = pd.read_csv('C:/Users/elysi/Tele2_project/data/geography/df_adm_distr.csv')

df_transport_agr.shape, df_adm_distr.shape

CPU times: total: 15.6 ms
Wall time: 23 ms


((3041, 7), (4768, 3))

In [547]:
for column in df_transport_agr.columns:
    if column not in ['h3_9_transport']:
        df_transport_agr.rename(columns={column: 'spb_trans_' + column}, inplace=True)

In [548]:
df_adm_distr = df_adm_distr[['h3_9_osm', 'name']]

**new geo features**

In [549]:
df_sport_final.shape,  df_cian_aggregated.shape,  df_metro_agr_neghbors.shape, 

((3905, 3), (6152, 7), (7755, 3))

In [550]:
df_yandex_unique_points_aggregated_binary.shape,  df_yandex_categories.shape,  df_yandex_sub_categories.shape, df_population_aggregated.shape

((7755, 5), (7755, 53), (7755, 26), (3650, 2))

**check cian nulls**

In [551]:
df_cian_aggregated.head(3)

,h3_9_yandex,cian_total_listing_count_including_hex_3hn,cian_total_area_including_hex_3hn,cian_median_area_including_hex_3hn,cian_avg_area_including_hex_3hn,cian_median_price_per_m2_including_hex_3hn,cian_avg_price_per_m2_including_hex_3hn
0,8911060557bffff,4.0,1029.3,244.00,221.433333,2169.98,2151.843333
1,8911060c3b7ffff,0.0,0.0,NaN,NaN,NaN,NaN
2,891106a8497ffff,7.0,1397.8,81.65,148.430000,1722.97,1656.990000


In [552]:
df_geo_spb = pd.merge(df_geo_spb, df_cian_aggregated,
                                how='left', left_on='h3_9', right_on='h3_9_yandex')

In [553]:
df_geo_spb.shape

(3892, 8)

In [554]:
(df_geo_spb.isna().sum() / len(df_geo_spb)).rename('percent of NA').to_frame().style.bar(vmax=1, vmin=0).format('{:.2%}')

,percent of NA
h3_9,0.00%
h3_9_yandex,2.44%
cian_total_listing_count_including_hex_3hn,2.44%
cian_total_area_including_hex_3hn,2.44%
cian_median_area_including_hex_3hn,5.16%
cian_avg_area_including_hex_3hn,5.16%
cian_median_price_per_m2_including_hex_3hn,5.27%
cian_avg_price_per_m2_including_hex_3hn,5.14%


In [555]:
df_geo_spb = df_geo_spb.drop(['h3_9_yandex'], axis=1)

In [556]:
df_geo_spb.shape

(3892, 7)

**join all geo features together**

In [557]:
#df_transport_agr - done
#df_adm_dist - done
#df_sport_final - done
#df_metro_agr_neghbors - done
#df_population_agr_updated - done
#df_cian_aggregated - done
#df_yandex_unique_points_aggregated_binary - done

#df_base_yandex_sub_categories
#df_base_yandex_sub_categories_neighbors

#df_yandex_categories
#df_base_yandex_categories_neighbors

In [558]:
df_geo_spb = pd.merge(df_geo_spb, df_adm_distr,
                                how='left', left_on='h3_9', right_on='h3_9_osm')

df_geo_spb = df_geo_spb.drop(['h3_9_osm'], axis=1)

In [559]:
df_geo_spb = pd.merge(df_geo_spb, df_transport_agr,
                                how='left', left_on='h3_9', right_on='h3_9_transport')

df_geo_spb = pd.merge(df_geo_spb, df_sport_final,
                                how='left', left_on='h3_9', right_on='h3_9')

df_geo_spb = pd.merge(df_geo_spb, df_metro_agr_neghbors,
                                how='left', left_on='h3_9', right_on='h3_9_metro')

df_geo_spb = pd.merge(df_geo_spb, df_population_agr_updated,
                                how='left', left_on='h3_9', right_on='h3_9_population')

df_geo_spb = pd.merge(df_geo_spb, df_base_yandex_categories_neighbors,
                                how='left', left_on='h3_9', right_on='h3_9')

df_geo_spb = pd.merge(df_geo_spb, df_base_yandex_sub_categories_neighbors,
                                how='left', left_on='h3_9', right_on='h3_9')

df_geo_spb = df_geo_spb.drop(['h3_9_transport', 'h3_9_metro', 'h3_9_population'], axis=1)

In [560]:
df_geo_spb = df_geo_spb.drop_duplicates(subset='h3_9', keep="first")

In [561]:
df_geo_spb.shape

(3892, 97)

In [562]:
(df_geo_spb.isna().sum() / len(df_geo_spb)).rename('percent of NA').to_frame().style.bar(vmax=1, vmin=0).format('{:.2%}')

,percent of NA
h3_9,0.00%
cian_total_listing_count_including_hex_3hn,2.44%
cian_total_area_including_hex_3hn,2.44%
cian_median_area_including_hex_3hn,5.16%
cian_avg_area_including_hex_3hn,5.16%
cian_median_price_per_m2_including_hex_3hn,5.27%
cian_avg_price_per_m2_including_hex_3hn,5.14%
name,0.00%
spb_trans_amount_unqiue_vehicles,49.15%
spb_trans_unique_vehicle_stops,49.15%


In [563]:
df_geo_spb = df_geo_spb.replace(np.nan, 0)

In [564]:
df_geo_spb.to_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_geo_spb_transport_fitnes_yandex_cian_metro_population_final_version_neighbors.csv', index=False)

**save binary unique features df**

In [573]:
df_yandex_unique_points_aggregated_binary.columns

Index(['total_subcategory_cinemas_neighbour_4_hn',
       'total_subcategory_shopping_centers_neighbour_4_hn',
       'total_subcategory_museums_neighbour_4_hn',
       'total_subcategory_parks_of_culture_and_leisure_neighbour_4_hn',
       'h3_9'],
      dtype='object')

In [574]:
rename_dict_subcategories = {
    'hex_id' : 'h3_9',
    'total_subcategory_cinemas_count_including_hex' : 'total_subcategory_cinemas_neighbour_4_hn',
    'total_subcategory_shopping_centers_count_including_hex' : 'total_subcategory_shopping_centers_neighbour_4_hn',
    'total_subcategory_museums_count_including_hex' : 'total_subcategory_museums_neighbour_4_hn',
    'total_subcategory_parks_of_culture_and_leisure_count_including_hex' : 'total_subcategory_parks_of_culture_and_leisure_neighbour_4_hn'
}

df_yandex_unique_points_aggregated_binary.rename(columns=rename_dict_subcategories, inplace=True)

In [575]:
df_yandex_unq_atr = df_geo_spb['h3_9']

In [576]:
df_yandex_unq_atr = pd.merge(df_yandex_unq_atr, df_yandex_unique_points_aggregated_binary,
                                how='left', left_on='h3_9', right_on='h3_9')

In [581]:
for column in df_yandex_unq_atr.columns:
    if column not in ['h3_9']:
        df_yandex_unq_atr.rename(columns={column: column + '_binary'}, inplace=True)

In [582]:
df_yandex_unq_atr.shape

(3892, 5)

In [583]:
df_yandex_unq_atr.head(5)

,h3_9,total_subcategory_cinemas_neighbour_4_hn_binary,total_subcategory_shopping_centers_neighbour_4_hn_binary,total_subcategory_museums_neighbour_4_hn_binary,total_subcategory_parks_of_culture_and_leisure_neighbour_4_hn_binary
0,8911060006bffff,0.0,1.0,0.0,1.0
1,89110600077ffff,0.0,1.0,0.0,0.0
2,89110600113ffff,0.0,1.0,0.0,0.0
3,89110600127ffff,0.0,0.0,0.0,0.0
4,89110600183ffff,1.0,1.0,0.0,0.0


In [584]:
(df_yandex_unq_atr.isna().sum() / len(df_yandex_unq_atr)).rename('percent of NA').to_frame().style.bar(vmax=1, vmin=0).format('{:.2%}')

,percent of NA
h3_9,0.00%
total_subcategory_cinemas_neighbour_4_hn_binary,2.44%
total_subcategory_shopping_centers_neighbour_4_hn_binary,2.44%
total_subcategory_museums_neighbour_4_hn_binary,2.44%
total_subcategory_parks_of_culture_and_leisure_neighbour_4_hn_binary,2.44%


In [585]:
df_yandex_unq_atr.to_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_yandex_unq_atr.csv', index=False)

## DF for clustering and SHAP values

In [586]:
df_geo_basic = pd.merge(df_geo_spb, df_yandex_unq_atr,
                                how='left', left_on='h3_9', right_on='h3_9')

In [587]:
df_geo_basic.shape

(3892, 101)

In [588]:
df_geo_basic.head(3)

,h3_9,cian_total_listing_count_including_hex_3hn,cian_total_area_including_hex_3hn,cian_median_area_including_hex_3hn,cian_avg_area_including_hex_3hn,cian_median_price_per_m2_including_hex_3hn,cian_avg_price_per_m2_including_hex_3hn,name,spb_trans_amount_unqiue_vehicles,spb_trans_unique_vehicle_stops,spb_trans_unique_stops,spb_trans_bus_stop_counter,spb_trans_tram_stop_counter,spb_trans_trolleybus_stop_counter,total_fit_clubs,total_fit_clubs_neighbour_2_hn,metro_count,total_metro_neighbour_5_hn,population_sum,total_population_neighbour_2_hn,category_auto_count_2_hn,category_security_count_2_hn,category_business_count_2_hn,category_government_count_2_hn,category_pets_count_2_hn,category_home_and_interior_count_2_hn,category_food_and_drinks_count_2_hn,category_computers_and_internet_count_2_hn,category_beauty_and_health_count_2_hn,category_culture_and_art_count_2_hn,category_stores_count_2_hn,category_medicine_and_pharmacy_count_2_hn,category_hospitals_count_2_hn,category_science_and_education_count_2_hn,category_kindergartens_count_2_hn,...,category_colleges_count_2_hn,category_lyceums_count_2_hn,category_general_education_schools_count_2_hn,category_planetariums_count_2_hn,category_amusement_parks_count_2_hn,category_industrial_enterprises_count_2_hn,subcategory_hospitals_count_2_hn,subcategory_kindergartens_count_2_hn,subcategory_universities_count_2_hn,subcategory_business_centers_count_2_hn,subcategory_dormitories_count_2_hn,subcategory_zoos_count_2_hn,subcategory_dolphinariums_and_aquariums_count_2_hn,subcategory_entertainment_centers_count_2_hn,subcategory_railway_stations_count_2_hn,subcategory_bus_stations_count_2_hn,subcategory_airports_count_2_hn,subcategory_hotels_count_2_hn,subcategory_parking_lots_and_garages_count_2_hn,subcategory_free_parking_count_2_hn,subcategory_parking_count_2_hn,subcategory_paid_parking_count_2_hn,subcategory_underground_parking_count_2_hn,subcategory_polyclinics_count_2_hn,subcategory_gymnasiums_count_2_hn,subcategory_colleges_count_2_hn,subcategory_lyceums_count_2_hn,subcategory_general_education_schools_count_2_hn,subcategory_planetariums_count_2_hn,subcategory_amusement_parks_count_2_hn,subcategory_industrial_enterprises_count_2_hn,total_subcategory_cinemas_neighbour_4_hn_binary,total_subcategory_shopping_centers_neighbour_4_hn_binary,total_subcategory_museums_neighbour_4_hn_binary,total_subcategory_parks_of_culture_and_leisure_neighbour_4_hn_binary
0,8911060006bffff,8.0,12062.9,1263.33,1287.046,1000.0,937.210,Пушкинский район,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,109.0,15.0,11.0,2.0,11.0,17.0,43.0,8.0,0.0,1.0,30.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,6.0,2.0,6.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
1,89110600077ffff,136.0,237533.7,951.95,1292.975,750.0,895.748,Московский район,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.0,12.0,7.0,1.0,2.0,0.0,32.0,12.0,0.0,1.0,10.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,89110600113ffff,10.0,16975.0,746.00,944.250,980.0,1390.000,Московский район,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,239.0,41.0,53.0,0.0,3.0,31.0,156.0,40.0,95.0,17.0,64.0,21.0,2.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,8.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,3.0,4.0,0.0,6.0,6.0,22.0,15.0,22.0,14.0,14.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0,0.0


In [589]:
df_geo_basic.to_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_geo_basic_clusters_shaps_final.csv', index=False)